# Making Folder for PNG's

In [1]:
import os

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
            #Test Line
            os.makedirs(os.path.join(directory, 'translated_files'))
            print('Folder Created')
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        
current_directory = os.getcwd()         
where_pdf_located = 'Untrans_pdf'
filename = "Monopoly 2"

#Location of where the png's will be
#desired_folder_location = path+'\\'+filename
desired_folder_location = os.path.join(current_directory, filename)

#desired_folder_name = path+'\\'+where_pdf_located+'\\'+filename
desired_folder_name = os.path.join(current_directory, where_pdf_located, filename)

createFolder(desired_folder_location)

Folder Created


# PDF to Image Splitter

In [2]:
import fitz
def convert_pdf_to_image(fic):
    #open your file
    doc = fitz.open(fic)
    #iterate through the pages of the document and create a RGB image of the page
    i = 0
    for page in doc:
        i+=1
        pix = page.get_pixmap()
        pix.save(f"{filename}\%i.png" % page.number)
        if i == 15:
            break
    print('PDF has been converted')
        
convert_pdf_to_image('Untrans_pdf'+'\\'+filename+'.pdf')

PDF has been converted


# Preprocessing Image Steps

In [3]:
#1.) make the image grayscaled
def grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def image_preprocessing(file_path):
    img = cv2.imread(file_path)
    gray_image = grayscale(img)
    #Actually making the image black and white
    thresh, img_bw = cv2.threshold(gray_image, 200, 230, cv2.THRESH_BINARY)
    return thresh, img_bw

#Dilation
def thick_font(image):
    import numpy as np
    image = cv2.bitwise_not(image)
    kernel = np.ones((1,1), np.uint8)
    #Thickening of text
    image = cv2.dilate(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return(image)

#Dilation
def thin_font(image):
    import numpy as np
    image = cv2.bitwise_not(image)
    kernel = np.ones((10,10), np.uint8)
    #Thinning of text
    image = cv2.erode(image, kernel, iterations=1)
    image = cv2.bitwise_not(image)
    return(image)

# Actually translating the PDF's

import os
from PIL import Image, ImageDraw, ImageFont
import textwrap
import cv2
from deep_translator import GoogleTranslator, single_detection
from pdf2image import convert_from_path
import easyocr
import math

font=r'C:\Users\pauln\Documents\Algo Trading\OCR\font\CC Wild Words Roman.ttf'

#easyOCR capturing text
image_location = desired_folder_location

#list_of_files = os.listdir(r"C:\Users\pauln\Documents\Lenovo Backups\Documents\Algo Trading\OCR\PDF_full\New folder")
list_of_files = os.listdir(desired_folder_location)

for j in range(len(list_of_files)-1):
    print(j)
    #for j in range(25, 30):
    if j%50==0 and j != 0:
        print(f"Page {j} is done")
        
#     #Testing
#     if j == 10:
#         break
        
    #Base Image
    mang_page = image_location+'\\'+str(j)+".png"
    
    #Processing the Image for OCR
    mang_page_processed = image_location+'\\'+str(j)+".png"
    
    img = cv2.imread(mang_page_processed)
    gray_image = grayscale(img)
    thresh, img_bw = cv2.threshold(gray_image, 200, 230, cv2.THRESH_BINARY)
    dilated_image = thick_font(img_bw)

#     in_line = True
#     reader = easyocr.Reader(['ch_sim'], gpu = True)
#     results = reader.readtext(dilated_image, paragraph=in_line)

    in_line = True
    reader = easyocr.Reader(['ch_sim'], gpu = True)
    results = reader.readtext(mang_page, paragraph=in_line)
    with Image.open(mang_page, mode='r') as im:
        try:
            for i in range(len(results)):
                captured_text = results[i][1]
                
                if len(captured_text) == 0:
                    break
                translated = GoogleTranslator(
                                    source='chinese (simplified)', 
                                    target='english').translate(captured_text)
                #print(translated)
                
                #Height and Weidth 
                height = abs(results[i][0][1][1] - results[i][0][2][1])
                width = abs(results[i][0][0][0] - results[i][0][1][0])

                #TopLeft and BottomRight Coordinates of rectangle to be drawn
                topLeft_X = results[i][0][0][0]
                topLeft_Y = results[i][0][0][1]

                botRight_X = results[i][0][2][0]
                botRight_Y = results[i][0][2][1]
                #print(topLeft_X,topLeft_Y,botRight_X,botRight_Y)
                #print(translated)
                selected_size = 0

                draw = ImageDraw.Draw(im)
                draw.rectangle((topLeft_X,topLeft_Y,botRight_X,botRight_Y), fill='white')

                for size in range(1,500):

                    font = ImageFont.truetype(
                            font=r'C:\Users\pauln\Documents\Algo Trading\OCR\font\CC Wild Words Roman.ttf', 
                            size=size)

                    text = translated
                    #print(text)
                    #print('Length of text: ' + str(len(text)) + '\n')
                    textBox_singlelined = font.getbbox(text)

                    single_Length = abs(textBox_singlelined[0] -  textBox_singlelined[2])
                    print(text)
                    print("Len Text: " + str(len(text)))
                    avg_char_width_using_getbbox = single_Length/len(text)

                    max_char_count_Using_getbbox = int((width * .95) / avg_char_width_using_getbbox )

                    text_wrapped = textwrap.fill(text=text, width=max_char_count_Using_getbbox)

                    current_lines = math.ceil(len(text)/max_char_count_Using_getbbox)

                    textBox_multilined = draw.multiline_textbbox(
                        xy=(im.size[0]/2, im.size[1]/2), 
                        text=text_wrapped, 
                        font=font, 
                        anchor='mm', 
                        align='center')

                    Multi_Length = abs(textBox_multilined[0] -  textBox_multilined[2])
                    Multi_Height = abs(textBox_multilined[1] - textBox_multilined[3])

                    height_PerLine = Multi_Height/current_lines

                    if Multi_Length>width*.95 and Multi_Height>height*.95:
                         break

                    selected_size+=1
                    
                font = ImageFont.truetype(
                            font=r'C:\Users\pauln\Documents\Algo Trading\OCR\font\CC Wild Words Roman.ttf', 
                            size=selected_size)

                #translated
                #print('Selected text: ' + text + '\n')
                #print('Length of text: ' + len(text))
                textBox_singlelined = font.getbbox(translated)

                single_Length = abs(textBox_singlelined[0] -  textBox_singlelined[2])
                avg_char_width_using_getbbox = single_Length/len(translated)

                max_char_count_Using_getbbox = int((width * .95) / avg_char_width_using_getbbox )

                text_wrapped = textwrap.fill(text=translated, width=max_char_count_Using_getbbox)
                draw.text(
                    xy=((results[i][0][0][0]+results[i][0][1][0])/2, (results[i][0][0][1]+results[i][0][2][1])/2), 
                    text=text_wrapped, 
                    font=font, 
                    fill='black', 
                    anchor='mm', 
                    align = 'center')

            print('Multi_Length: ' + str(Multi_Length))
            print('Length of rectangle: ' + str(width))
            print('Multi_Height: ' + str(Multi_Height))
            print('Height of rectang;e: ' + str(height))
        except TypeError:
            print('Error occured ¯\_(ツ)_/¯' + '\n')
            im.save(desired_folder_location+'\\translated_files\\'+str(j)+'.pdf')
            print(f'Page {j} Saved' + '\n')
            continue
        im.save(desired_folder_location+'\\translated_files\\'+str(j)+'.pdf')
        print(f'Page {j} Saved' + '\n')

In [6]:
import os
from PIL import Image, ImageDraw, ImageFont
import textwrap
import cv2
from deep_translator import GoogleTranslator, single_detection
from pdf2image import convert_from_path
import easyocr
import math

font=r'C:\Users\pauln\Documents\Algo Trading\OCR\font\CC Wild Words Roman.ttf'

#easyOCR capturing text
image_location = desired_folder_location

#list_of_files = os.listdir(r"C:\Users\pauln\Documents\Lenovo Backups\Documents\Algo Trading\OCR\PDF_full\New folder")
list_of_files = os.listdir(desired_folder_location)

for j in range(len(list_of_files)-1):
    print(j)
    #for j in range(25, 30):
    if j%50==0 and j != 0:
        print(f"Page {j} is done")
        
#     #Testing
#     if j == 10:
#         break
        
    #Base Image
    mang_page = image_location+'\\'+str(j)+".png"
    
    #Processing the Image for OCR
    mang_page_processed = image_location+'\\'+str(j)+".png"

#     in_line = True
#     reader = easyocr.Reader(['ch_sim'], gpu = True)
#     results = reader.readtext(dilated_image, paragraph=in_line)

    in_line = True
    reader = easyocr.Reader(['ch_sim'], gpu = True)
    results = reader.readtext(mang_page, paragraph=in_line)
    
    with Image.open(mang_page) as im:
        for i in range(len(results)):
            try:
                captured_text = results[i][1]

                #if len(captured_text) == 0:
                    #break
                translated = GoogleTranslator(
                                    source='chinese (simplified)', 
                                    target='english').translate(captured_text)
                print(translated)

                #Height and Weidth 
                height = abs(results[i][0][1][1] - results[i][0][2][1])
                width = abs(results[i][0][0][0] - results[i][0][1][0])

                #TopLeft and BottomRight Coordinates of rectangle to be drawn
                topLeft_X = results[i][0][0][0]
                topLeft_Y = results[i][0][0][1]

                botRight_X = results[i][0][2][0]
                botRight_Y = results[i][0][2][1]

                selected_size = 0

                draw = ImageDraw.Draw(im)
                draw.rectangle((topLeft_X,topLeft_Y,botRight_X,botRight_Y), fill='white')
            except:
                continue
            for size in range(1,500):
                try:
                    font = ImageFont.truetype(
                            font=r'C:\Users\pauln\Documents\Algo Trading\OCR\font\CC Wild Words Roman.ttf', 
                            size=size)

                    text = translated

                    textBox_singlelined = font.getbbox(text)

                    single_Length = abs(textBox_singlelined[0] -  textBox_singlelined[2])
                    avg_char_width_using_getbbox = single_Length/len(text)

                    max_char_count_Using_getbbox = int((width * .95) / avg_char_width_using_getbbox )

                    text_wrapped = textwrap.fill(text=text, width=max_char_count_Using_getbbox)

                    current_lines = math.ceil(len(text)/max_char_count_Using_getbbox)

                    textBox_multilined = draw.multiline_textbbox(
                        xy=(im.size[0]/2, im.size[1]/2), 
                        text=text_wrapped, 
                        font=font, 
                        anchor='mm', 
                        align='center')

                    Multi_Length = abs(textBox_multilined[0] -  textBox_multilined[2])
                    Multi_Height = abs(textBox_multilined[1] - textBox_multilined[3])

                    height_PerLine = Multi_Height/current_lines

                    if Multi_Length>width*.9 and Multi_Height>height*.9:
                         break

                    selected_size+=1
                except:
                    break
            try:
                font = ImageFont.truetype(
                            font=r'C:\Users\pauln\Documents\Algo Trading\OCR\font\CC Wild Words Roman.ttf', 
                            size=selected_size)

                textBox_singlelined = font.getbbox(translated)

                single_Length = abs(textBox_singlelined[0] -  textBox_singlelined[2])
                avg_char_width_using_getbbox = single_Length/len(translated)

                max_char_count_Using_getbbox = int((width * .95) / avg_char_width_using_getbbox )

                text_wrapped = textwrap.fill(text=translated, width=max_char_count_Using_getbbox)
                draw.text(
                    xy=((results[i][0][0][0]+results[i][0][1][0])/2, (results[i][0][0][1]+results[i][0][2][1])/2), 
                    text=text_wrapped, 
                    font=font, 
                    fill='black', 
                    anchor='mm', 
                    align = 'center')
            except:
                continue
        im.save(desired_folder_location+'\\translated_files\\'+str(j)+'.pdf')
        print(f'Page {j} Saved' + '\n')

0
"Sexy Monopoly" super fine and super long - 2 3 %
290?8@
cry
husband
None
Page 0 Saved

1
When Mengyao comes out again. The bondage and semen all over her body disappeared. But she also staggered and almost fell
Huh, it finally came out
Page 1 Saved

2
Boss Liu couldn't help being surprised to see that Mengyao appeared in her original position, but her body was still intact
players on the playing field
The state at the end of the scene will not be brought back to reality, but the memories and feelings in the scene will not be forgotten
Huh? Mengyao was covered in semen just now?
Page 2 Saved

3
This setting is interesting
That's right. Otherwise, the game will be disabled after a few rounds. How can we continue the game?
Page 3 Saved

4
Mengyao stood up after a while, and immediately a staff member stepped forward to hook her up again, and continued the competition
Page 4 Saved

5
After this round. Mengyao's score also became the lowest among the crowd 78000 81 96000

Name: Wan'er Me

# Merging files

In [7]:
import os
name = f'C:\\Users\\pauln\\Documents\\Algo Trading\\OCR\\PDF_full\\{filename}\\translated_files'
#name = f'C:\\Users\\pauln\\Documents\\Lenovo Backups\\Documents\\Algo Trading\\OCR\\PDF_full\\{filename}\\translated_files'
source_dir = name
desired_filename = filename

def merge_pdf(path_to_pdf, desired_filename):
    from PyPDF2 import PdfFileMerger

    merger = PdfFileMerger()
    source_dir = path_to_pdf
    list_of_files = os.listdir(source_dir)
    
    file_limit = 500000000
    current_file_size = 0
    number = 0

    for i in range(len(list_of_files)):
        current_file_size+=os.path.getsize(source_dir+'\\'+str(i)+'.pdf')
        if file_limit > current_file_size:
            merger.append(source_dir+'\\'+str(i)+'.pdf')
            
        else:
            current_file_size = os.path.getsize(source_dir+'\\'+str(i)+'.pdf')
            merger.write('Combined_translated'+'\\'+desired_filename+f' {str(number)}.pdf')
            print('First Set Done')
            number += 1
            merger = PdfFileMerger()
            merger.append(source_dir+'\\'+str(i)+'.pdf')

            

    merger.write('Combined_translated'+'\\'+desired_filename+f' {str(number)}.pdf')

    print('Pages Merged!')
    
merge_pdf(source_dir, desired_filename)

Pages Merged!


## Error Testing

In [8]:
import os
from PIL import Image, ImageDraw, ImageFont
import textwrap
import cv2
from deep_translator import GoogleTranslator, single_detection
from pdf2image import convert_from_path
import easyocr
import math
font=r'C:\Users\pauln\Documents\Algo Trading\OCR\font\CC Wild Words Roman.ttf'
in_line = True
test_image = r'C:\Users\pauln\Documents\Algo Trading\OCR\PDF_full\Monopoly 2\15.png'
#test_image = r'C:\Users\pauln\Documents\Algo Trading\OCR\PDF_full\Meng Yao 13\2.png'

reader = easyocr.Reader(['ch_sim'], gpu = True)
results = reader.readtext(test_image, paragraph=in_line)
#print(results)

with Image.open(test_image) as im:
    for i in range(len(results)):
        try:
            captured_text = results[i][1]

            translated = GoogleTranslator(
                                source='chinese (simplified)', 
                                target='english').translate(captured_text)
            print(translated)
            #Height and Weidth 
            height = abs(results[i][0][1][1] - results[i][0][2][1])
            width = abs(results[i][0][0][0] - results[i][0][1][0])

            #TopLeft and BottomRight Coordinates of rectangle to be drawn
            topLeft_X = results[i][0][0][0]
            topLeft_Y = results[i][0][0][1]

            botRight_X = results[i][0][2][0]
            botRight_Y = results[i][0][2][1]
            #print(topLeft_X,topLeft_Y,botRight_X,botRight_Y)
            #print(translated)
            selected_size = 0

            draw = ImageDraw.Draw(im)
            draw.rectangle((topLeft_X,topLeft_Y,botRight_X,botRight_Y), fill='white')
        except:
            continue
        for size in range(1,500):
            try:
                print('Size:' + str(size))
                font = ImageFont.truetype(
                        font=r'C:\Users\pauln\Documents\Algo Trading\OCR\font\CC Wild Words Roman.ttf', 
                        size=size)

                text = translated
                textBox_singlelined = font.getbbox(text)

                single_Length = abs(textBox_singlelined[0] -  textBox_singlelined[2])
                avg_char_width_using_getbbox = single_Length/len(text)

                max_char_count_Using_getbbox = int((width * .95) / avg_char_width_using_getbbox )

                text_wrapped = textwrap.fill(text=text, width=max_char_count_Using_getbbox)

                current_lines = math.ceil(len(text)/max_char_count_Using_getbbox)

                textBox_multilined = draw.multiline_textbbox(
                    xy=(im.size[0]/2, im.size[1]/2), 
                    text=text_wrapped, 
                    font=font, 
                    anchor='mm', 
                    align='center')
                
                print('Width: ' + str(width))
                print('Height: ' + str(height))
                
                Multi_Length = abs(textBox_multilined[0] -  textBox_multilined[2])
                print('Prog Multi_Length: ' + str(Multi_Length))
                Multi_Height = abs(textBox_multilined[1] - textBox_multilined[3])
                print('Prog Multi_Height: ' + str(Multi_Height))
                height_PerLine = Multi_Height/current_lines
                if Multi_Length>width*.95 and Multi_Height>height*.9:
                     break

                selected_size+=1
            except:
                break
        try:
            font = ImageFont.truetype(
                        font=r'C:\Users\pauln\Documents\Algo Trading\OCR\font\CC Wild Words Roman.ttf', 
                        size=selected_size)

            text = translated
            textBox_singlelined = font.getbbox(text)
            print('\n' + 'Chosen Size: ' + str(size))
            print(text)
            print(textBox_singlelined)
            #print('\n')
            print('Width: ' + str(width))
            print('Height: ' + str(height))

            single_Length = abs(textBox_singlelined[0] -  textBox_singlelined[2])
            avg_char_width_using_getbbox = single_Length/len(text)

            max_char_count_Using_getbbox = int((width * .95) / avg_char_width_using_getbbox )

            text_wrapped = textwrap.fill(text=text, width=max_char_count_Using_getbbox)
            draw.text(
                xy=((results[i][0][0][0]+results[i][0][1][0])/2, (results[i][0][0][1]+results[i][0][2][1])/2), 
                text=text_wrapped, 
                font=font, 
                fill='black', 
                anchor='mm', 
                align = 'center')
            
            textBox_multilined = draw.multiline_textbbox(
                    xy=(im.size[0]/2, im.size[1]/2), 
                    text=text_wrapped, 
                    font=font, 
                    anchor='mm', 
                    align='center')

            Multi_Length = abs(textBox_multilined[0] -  textBox_multilined[2])
            print('Multi_Length: ' + str(Multi_Length))
            Multi_Height = abs(textBox_multilined[1] - textBox_multilined[3])
            print('Multi_Height: ' + str(Multi_Height))
            height_PerLine = Multi_Height/current_lines
            
            
            
        except:
            continue
        #im.save('C:\\Users\\pauln\\Documents\\Algo Trading\\OCR\\PDF_full\\blank_error.png')
im.show()


Here is another umbrella to choose to go to} "The small grid next to the grid is basically full of props. Every time the contestants take a step, there is a high probability that they will enter the grid with props-
Size:1
Width: 2719
Height: 152
Prog Multi_Length: 159.0
Prog Multi_Height: 3.0
Size:2
Width: 2719
Height: 152
Prog Multi_Length: 213.0
Prog Multi_Height: 3.0
Size:3
Width: 2719
Height: 152
Prog Multi_Length: 376.0
Prog Multi_Height: 4.0
Size:4
Width: 2719
Height: 152
Prog Multi_Length: 512.0
Prog Multi_Height: 5.0
Size:5
Width: 2719
Height: 152
Prog Multi_Length: 624.0
Prog Multi_Height: 6.0
Size:6
Width: 2719
Height: 152
Prog Multi_Length: 748.0
Prog Multi_Height: 6.0
Size:7
Width: 2719
Height: 152
Prog Multi_Length: 872.0
Prog Multi_Height: 7.0
Size:8
Width: 2719
Height: 152
Prog Multi_Length: 994.0
Prog Multi_Height: 9.0
Size:9
Width: 2719
Height: 152
Prog Multi_Length: 1101.0
Prog Multi_Height: 10.0
Size:10
Width: 2719
Height: 152
Prog Multi_Length: 1238.0
Prog Multi_He